### Apply EM Algorithm to impute missing values
Ref: https://joon3216.github.io/research_materials/2019/em_imputation_python.html

In [11]:
from datetime import datetime as dt
import numpy as np
from functools import reduce

####  Introduction
This note is about replicating R functions written in Imputing missing data using EM algorithm under 2019: Methods for Multivariate Data. simulate_na (which will be renamed as simulate_nan here) and impute_em are going to be written in Python, and the computation time of impute_em will be checked in both Python and R.

#### Functions
All the details of how the algorithm works is presented here.

According to the following pandas documentation, missing values in Python are denoted as:

NaN in numeric arrays
None or NaN in object arrays
NaT in datetimelike
In this note, I will use np.nan to denote missing components since we are dealing with numeric arrays.

In [2]:
np.random.seed(1024)
mu = np.array([1, 2, 6])
Sigma = np.array([[118, 62, 44], [62, 49, 17], [44, 17, 21]])
n = 400
X_truth = np.random.multivariate_normal(mu, Sigma, n)

#### Simulating np.nan’s
Simulating np.nan’s in a numeric array can be done using the same workflow as in simulate_na:

In [3]:
def simulate_nan(X, nan_rate):
    '''(np.array, number) -> {str: np.array or number}
    
    Preconditions:
    1. np.isnan(X_complete).any() == False
    2. 0 <= nan_rate <= 1
    
    Return the dictionary with four keys where: 
    - Key 'X' stores a np.array where some of the entries in X 
      are replaced with np.nan based on nan_rate specified.
    - Key 'C' stores a np.array where each entry is False if the
      corresponding entry in the key 'X''s np.array is np.nan, and True
      otherwise.
    - Key 'nan_rate' stores nan_rate specified.
    - Key 'nan_rate_actual' stores the actual proportion of np.nan
      in the key 'X''s np.array.
    '''
    
    # Create C matrix; entry is False if missing, and True if observed
    X_complete = X.copy()
    nr, nc = X_complete.shape
    C = np.random.random(nr * nc).reshape(nr, nc) > nan_rate
    
    # Check for which i's we have all components become missing
    checker = np.where(sum(C.T) == 0)[0]
    if len(checker) == 0:
        # Every X_i has at least one component that is observed,
        # which is what we want
        X_complete[C == False] = np.nan
    else:
        # Otherwise, randomly "revive" some components in such X_i's
        for index in checker:
            reviving_components = np.random.choice(
                nc, 
                int(np.ceil(nc * np.random.random())), 
                replace = False
            )
            C[index, np.ix_(reviving_components)] = True
        X_complete[C == False] = np.nan
    
    result = {
        'X': X_complete,
        'C': C,
        'nan_rate': nan_rate,
        'nan_rate_actual': np.sum(C == False) / (nr * nc)
    }
    
    return result

In [6]:
result = simulate_nan(X_truth, nan_rate = .4)
X = result['X'].copy()
X

array([[-22.51504305, -11.21931542,  -0.31680441],
       [ -4.13801604,  -3.3813311 ,          nan],
       [         nan,          nan,   1.21653198],
       ...,
       [         nan,          nan,   3.44262394],
       [  1.37769221,          nan,   5.10726117],
       [         nan,          nan,  11.05380599]])

In [7]:
result['nan_rate_actual']

0.3525

#### Imputing np.nan’s
In Python, impute_em can be written as follows:

In [10]:
def impute_em(X, max_iter = 3000, eps = 1e-08):
    '''(np.array, int, number) -> {str: np.array or int}
    
    Precondition: max_iter >= 1 and eps > 0
    
    Return the dictionary with five keys where:
    - Key 'mu' stores the mean estimate of the imputed data.
    - Key 'Sigma' stores the variance estimate of the imputed data.
    - Key 'X_imputed' stores the imputed data that is mutated from X using 
      the EM algorithm.
    - Key 'C' stores the np.array that specifies the original missing entries
      of X.
    - Key 'iteration' stores the number of iteration used to compute
      'X_imputed' based on max_iter and eps specified.
    '''
    
    nr, nc = X.shape
    C = np.isnan(X) == False
    
    # Collect M_i and O_i's
    one_to_nc = np.arange(1, nc + 1, step = 1)
    M = one_to_nc * (C == False) - 1
    O = one_to_nc * C - 1
    
    # Generate Mu_0 and Sigma_0
    Mu = np.nanmean(X, axis = 0)
    observed_rows = np.where(np.isnan(sum(X.T)) == False)[0]
    S = np.cov(X[observed_rows, ].T)
    if np.isnan(S).any():
        S = np.diag(np.nanvar(X, axis = 0))
    
    # Start updating
    Mu_tilde, S_tilde = {}, {}
    X_tilde = X.copy()
    no_conv = True
    iteration = 0
    while no_conv and iteration < max_iter:
        for i in range(nr):
            S_tilde[i] = np.zeros(nc ** 2).reshape(nc, nc)
            if set(O[i, ]) != set(one_to_nc - 1): # missing component exists
                M_i, O_i = M[i, ][M[i, ] != -1], O[i, ][O[i, ] != -1]
                S_MM = S[np.ix_(M_i, M_i)]
                S_MO = S[np.ix_(M_i, O_i)]
                S_OM = S_MO.T
                S_OO = S[np.ix_(O_i, O_i)]
                Mu_tilde[i] = Mu[np.ix_(M_i)] +\
                    S_MO @ np.linalg.inv(S_OO) @\
                    (X_tilde[i, O_i] - Mu[np.ix_(O_i)])
                X_tilde[i, M_i] = Mu_tilde[i]
                S_MM_O = S_MM - S_MO @ np.linalg.inv(S_OO) @ S_OM
                S_tilde[i][np.ix_(M_i, M_i)] = S_MM_O
        Mu_new = np.mean(X_tilde, axis = 0)
        S_new = np.cov(X_tilde.T, bias = 1) +\
            reduce(np.add, S_tilde.values()) / nr
        no_conv =\
            np.linalg.norm(Mu - Mu_new) >= eps or\
            np.linalg.norm(S - S_new, ord = 2) >= eps
        Mu = Mu_new
        S = S_new
        iteration += 1
    
    result = {
        'mu': Mu,
        'Sigma': S,
        'X_imputed': X_tilde,
        'C': C,
        'iteration': iteration
    }
    
    return result

In [12]:
start = dt.now()
result_imputed = impute_em(X)
end = dt.now()

In [13]:
result_imputed['mu'] # estimate using the imputed data

array([1.52026248, 1.70293513, 6.36332455])

In [14]:
result_imputed['Sigma'] # estimate using the imputed data

array([[120.74128223,  68.50401933,  43.91343725],
       [ 68.50401933,  54.44391386,  19.22271582],
       [ 43.91343725,  19.22271582,  20.2735122 ]])

In [15]:
Sigma # truth

array([[118,  62,  44],
       [ 62,  49,  17],
       [ 44,  17,  21]])

The imputation is done as follows:

In [16]:
X[np.arange(0, 9), ] # data with missing components

array([[-22.51504305, -11.21931542,  -0.31680441],
       [ -4.13801604,  -3.3813311 ,          nan],
       [         nan,          nan,   1.21653198],
       [  6.57201047,   6.0520226 ,   8.87408451],
       [         nan,   3.46018924,   9.17663177],
       [ -2.71777693,   1.51178112,   4.17435875],
       [         nan,  -5.44848469,          nan],
       [ -4.17891721,  -3.48215875,   4.29849034],
       [         nan,   1.28621915,   0.98402706]])

In [17]:
result_imputed['X_imputed'][np.arange(0, 9), ] # imputed data

array([[-22.51504305, -11.21931542,  -0.31680441],
       [ -4.13801604,  -3.3813311 ,   4.99018376],
       [ -9.62794659,  -3.17709404,   1.21653198],
       [  6.57201047,   6.0520226 ,   8.87408451],
       [  6.9388001 ,   3.46018924,   9.17663177],
       [ -2.71777693,   1.51178112,   4.17435875],
       [ -7.47800687,  -5.44848469,   3.83834607],
       [ -4.17891721,  -3.48215875,   4.29849034],
       [ -6.65708132,   1.28621915,   0.98402706]])

#### My Data

In [24]:
X2 = np.array([[np.nan, 6, 3, 4, np.nan], [1, np.nan, 4, np.nan, np.nan], 
          [3, 7, 5, 6, 7], [2, 8, 2, np.nan, np.nan]])
X2

array([[nan,  6.,  3.,  4., nan],
       [ 1., nan,  4., nan, nan],
       [ 3.,  7.,  5.,  6.,  7.],
       [ 2.,  8.,  2., nan, nan]])

In [30]:
result_imputed2 = impute_em(X2)

/home/rossleecooloh/.pyenv/versions/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: Degrees of freedom <= 0 for slice


In [31]:
result_imputed2['X_imputed']

array([[0.69678127, 6.        , 3.        , 4.        , 7.        ],
       [1.        , 5.63599795, 4.        , 4.36635373, 7.        ],
       [3.        , 7.        , 5.        , 6.        , 7.        ],
       [2.        , 8.        , 2.        , 4.83346284, 7.        ]])